In [2]:
from sys import exit
from Crypto.Util.number import bytes_to_long, inverse, long_to_bytes
import random
import string
from pwn import *
CHARSET = string.ascii_uppercase + string.ascii_lowercase + string.digits

## **Bài toán GUESS MY CHEESE (Part 2)**

Bài toán đưa ra một list danh sách các mật khẩu có thể, sau đó, đưa ra 3 gợi ý, từ các gợi ý ta có thể suy ra được:
- Mật khẩu trước tiên được thêm 2 byte ngẫu nhiên vào password
- Sau đó, sử dụng hàm băm SHA-256 để băm

==> Mục tiêu là lấy mã băm và so khớp, do gợi ý thứ 3 yêu cầu ta dùng "Rainbow table" để so khớp mã chính xác. Ta thực hiện thuật toán như sau:

In [12]:
import hashlib
import itertools

def find_matching_password(file_path, target_hash):
    salts = [f"{i:02x}" for i in range(256)]  # Salt từ "00" đến "ff"

    with open(file_path, "r") as file:
        passwords = [line.strip() for line in file.readlines()]

    for password in passwords:
        for salt in salts:
            for i in range(len(password) + 1):  # Thử đặt salt ở mọi vị trí từ 0 đến cuối
                salted_password = password[:i] + salt + password[i:]
                hash_value = hashlib.sha256((salted_password).encode('utf-8')).hexdigest()
                # print(hash_value)
                if hash_value[:10] == target_hash[:10]:
                    return password, salt

    return None  

def find_matching_password2(file_path, target_hash):
    salts = [f"{i:02x}" for i in range(256)]  # Tạo danh sách salt từ "00" đến "ff"
    print(len(salts))
    
    with open(file_path, "r") as file:
        passwords = [line.strip() for line in file.readlines()]

    for password in passwords:
        positions = list(itertools.combinations(range(len(password) + 2), 2))  # Tạo các cặp vị trí chèn salt

        for salt in salts:  # Duyệt qua tất cả salt có thể
            for pos1, pos2 in positions:  # Duyệt qua tất cả vị trí chèn salt
                salted_password = (
                    password[:pos1] + salt[0] +  # Chèn ký tự đầu tiên của salt
                    password[pos1:pos2] + salt[1] +  # Chèn ký tự thứ hai của salt
                    password[pos2:]
                )

                hash_value = hashlib.sha256(salted_password.encode('utf-8')).hexdigest()

                if hash_value[:10] == target_hash[:10]:  # So sánh 10 ký tự đầu của hash
                    return password, salt, pos1, pos2

    return None


file_path = "cheese_list.txt"
target_hash = "b13b96db08d3c9d38b555ace3d12434617fd4dd5f45600a106aa32e9b05fa876"
matched_password = find_matching_password(file_path, target_hash)

if matched_password:
    print(f"Found matching password: {matched_password}")
else:
    print("No matching password found.")
    
matched_password = find_matching_password2(file_path, target_hash)
if matched_password:
    print(f"Found matching password: {matched_password}")
else:
    print("No matching password found.")

No matching password found.
256
No matching password found.


'6f58907a60a6103afa876b0f25ad2c2e9b62693e499d7da7fdbd3b4ca77172fe'